In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
!tar zxvf /content/drive/My\ Drive/Colab\ Notebooks/train.tgz
!pip install wandb

In [0]:
!wandb login

# CAPTCHA Recognition - 6 digits

In [0]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, Conv2D, Flatten, Dropout, MaxPooling2D,BatchNormalization

import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd



# Parsing Data

In [0]:
CLASS_NAMES = np.array([])
for i in range(10):
    CLASS_NAMES =np.append(CLASS_NAMES,chr(ord("0")+i))
for i in range(26):
    CLASS_NAMES = np.append(CLASS_NAMES,chr(ord("A")+i))

In [0]:
#functions
def parse_label(strIN):
    return (strIN==CLASS_NAMES).astype(float)
def readimg_to_tensor(fn):
    a = tf.io.read_file((fn))
        #crop to [60,30,3]
    img = tf.io.decode_jpeg(a)
    return img

In [0]:
#training data 01
def prepData01():  
  train_dir_data01 = os.path.abspath(os.getcwd())+"/train/data01_train/"
  data = pd.read_csv("./train/data01_train.csv")
  for i in range(len(data)):
      data.iloc[i,1] = list(data.iloc[i,1])
  arr = np.zeros([6,50000],str)
  for j in range(6):
      for i in range(len(data)):
          arr[j][i] = data.iloc[i,1][j]
  data = data.drop(columns= ["code"]).join(pd.DataFrame(arr.transpose(),columns = ["code0","code1",'code2','code3','code4','code5']))

  for i in range(6):
      data.iloc[:,i+1] = data.iloc[:,i+1].map(parse_label)

  TL = tf.convert_to_tensor(list(data.iloc[:,1].values))
  TL2 = tf.convert_to_tensor(list(data.iloc[:,2].values))
  TL3 = tf.convert_to_tensor(list(data.iloc[:,3].values))
  TL4 = tf.convert_to_tensor(list(data.iloc[:,4].values))
  TL5 = tf.convert_to_tensor(list(data.iloc[:,5].values))
  TL6 = tf.convert_to_tensor(list(data.iloc[:,6].values))

  train_dir = tf.data.Dataset.list_files(train_dir_data01+'*.jpg',shuffle=False) 
  train_data = train_dir.map(lambda x: readimg_to_tensor(x))
  train_label = tf.data.Dataset.from_tensor_slices((TL,TL2,TL3,TL4,TL5,TL6))
  train_data01=tf.data.Dataset.zip((train_data,train_label))
  validation_split=0.1
  split = int(num_elements*validation_split)
  train_data_gen=iter(train_data01.shuffle(50000,seed = 102).batch(50000))

  img , lb = next(train_data_gen)
  img_train = img[:-split]
  lb_train = (lb[0][:-split],lb[1][:-split],lb[2][:-split],lb[3][:-split],lb[4][:-split],lb[5][:-split])
  img_test = img[-split:]
  lb_test = (lb[0][-split:],lb[1][-split:],lb[2][-split:],lb[3][-split:],lb[4][-split:],lb[5][-split:])
  return img_train, lb_train, img_test, lb_test

In [0]:
#training data 02
def prepData02(): 
  train_dir_data02 = os.path.abspath(os.getcwd())+"/train/data02_train/"
  data2 = pd.read_csv("./train/data02_train.csv")
  for i in range(len(data2)):
      data2.iloc[i,1] = list(data2.iloc[i,1])
  arr = np.zeros([6,50000],str)
  for j in range(6):
      for i in range(len(data2)):
          arr[j][i] = data2.iloc[i,1][j]
  data2 = data2.drop(columns= ["code"]).join(pd.DataFrame(arr.transpose(),columns = ["code0","code1",'code2','code3','code4','code5']))

  #to one hot
  for i in range(6):
      data2.iloc[:,i+1] = data2.iloc[:,i+1].map(parse_label)

  TL = tf.convert_to_tensor(list(data2.iloc[:,1].values))
  TL2 = tf.convert_to_tensor(list(data2.iloc[:,2].values))
  TL3 = tf.convert_to_tensor(list(data2.iloc[:,3].values))
  TL4 = tf.convert_to_tensor(list(data2.iloc[:,4].values))
  TL5 = tf.convert_to_tensor(list(data2.iloc[:,5].values))
  TL6 = tf.convert_to_tensor(list(data2.iloc[:,6].values))

  train_dir2 = tf.data.Dataset.list_files(train_dir_data02+'*.jpg',shuffle=False) 
  train_data2 = train_dir2.map(lambda x: readimg_to_tensor(x))
  train_label2 = tf.data.Dataset.from_tensor_slices((TL,TL2,TL3,TL4,TL5,TL6))
  train_data02=tf.data.Dataset.zip((train_data2,train_label2))
  validation_split=0.1
  num_elements=50000
  split = int(num_elements*validation_split)
  train_data_gen=iter(train_data02.shuffle(50000,seed = 1).batch(50000))

  img , lb = next(train_data_gen)
  img_train = img[:-split]
  lb_train = (lb[0][:-split],lb[1][:-split],lb[2][:-split],lb[3][:-split],lb[4][:-split],lb[5][:-split])
  img_test = img[-split:]
  lb_test = (lb[0][-split:],lb[1][-split:],lb[2][-split:],lb[3][-split:],lb[4][-split:],lb[5][-split:])
  return img_train, lb_train, img_test, lb_test

In [0]:
#choose data
img_train, lb_train, img_test, lb_test= prepData01()

# Model configeration

In [0]:
import wandb
from tensorflow.keras import layers
from tensorflow.keras import backend

In [0]:
filepath = "/content/drive/Shared drives/TingWeis_Drive/Colab Notebooks/Saved Model/"#"h5s/"

In [0]:
config_default = {
    "weight_regu":1e-4,
    "LR":1e-3,
    "dropout":0.3,
    "name":"Mymodel",
    "batch":256
}

In [0]:
def block2(x, filters, kernel_size=3, stride=1, conv_shortcut=True, name=None, config = config_default):
  """A residual block.
  Arguments:
      x: input tensor.
      filters: integer, filters of the bottleneck layer.
      kernel_size: default 3, kernel size of the bottleneck layer.
      stride: default 1, stride of the first layer.
      conv_shortcut: default False, use convolution shortcut if True,
        otherwise identity shortcut.
      name: string, block label.
  Returns:
    Output tensor for the residual block.
  """
  initializer = tf.keras.initializers.he_normal( seed = 3)
  alpha = config["weight_regu"]  # weight decay coefficient
  regularizer = tf.keras.regularizers.l2(alpha)
  bn_axis = 3 if backend.image_data_format() == 'channels_last' else 1

  preact = layers.BatchNormalization(epsilon=1.001e-5, name=name + '_preact_bn')(x)
  preact = layers.Activation('relu', name=name + '_preact_relu')(preact)

  if conv_shortcut:
    shortcut = layers.Conv2D(
        4 * filters, 1, strides=stride, name=name + '_0_conv', kernel_initializer = initializer,kernel_regularizer = regularizer)(preact)
  else:
    shortcut = layers.MaxPooling2D(1, strides=stride)(x) if stride > 1 else x

  x = layers.Conv2D(
      filters, 1, strides=1, use_bias=False, name=name + '_1_conv', kernel_initializer = initializer,kernel_regularizer = regularizer)(preact)
  x = layers.BatchNormalization(epsilon=1.001e-5, name=name + '_1_bn')(x)
  x = layers.Activation('relu', name=name + '_1_relu')(x)

  x = layers.ZeroPadding2D(padding=((1, 1), (1, 1)), name=name + '_2_pad')(x)
  x = layers.Conv2D(
      filters,
      kernel_size,
      strides=stride,
      use_bias=False,
      name=name + '_2_conv', kernel_initializer = initializer,kernel_regularizer = regularizer)(x)
  x = layers.BatchNormalization(epsilon=1.001e-5, name=name + '_2_bn')(x)
  x = layers.Activation('relu', name=name + '_2_relu')(x)

  x = layers.Conv2D(4 * filters, 1, name=name + '_3_conv', kernel_initializer = initializer,kernel_regularizer = regularizer)(x)
  x = layers.Dropout(config["dropout"])(x)
  x = layers.Add(name=name + '_out')([shortcut, x])
  return x

def stack1(x, filters, blocks, stride1=2, name=None):
  """A set of stacked residual blocks.
  Arguments:
    x: input tensor.
    filters: integer, filters of the bottleneck layer in a block.
    blocks: integer, blocks in the stacked blocks.
    stride1: default 2, stride of the first layer in the first block.
    name: string, stack label.
  Returns:
    Output tensor for the stacked blocks.
  """
  x = block2(x, filters, conv_shortcut=True ,stride=stride1, name=name + '_block1')
  for i in range(2, blocks + 1):
    x = block2(x, filters, conv_shortcut=False, name=name + '_block' + str(i))
  return x

In [0]:
def build_ResNet_Model(config = config_default):
  initializer = tf.keras.initializers.he_normal( seed = 3)
  alpha = config["weight_regu"]  # weight decay coefficient
  regularizer = tf.keras.regularizers.l2(alpha)
  def stack_fn(x):
    x = stack1(x, 32, 2, stride1=1, name='conv2')
    x = stack1(x, 64, 2, name='conv3')
    x = stack1(x, 128, 2, name='conv4')
    return stack1(x, 256, 2, name='conv5')
  #input shape
  img_input = layers.Input(shape=(60,200,3))

  
  x = layers.Conv2D(32, 3, strides=1,padding="same", name='conv1_conv',kernel_initializer = initializer,kernel_regularizer = regularizer)(img_input)
  #pre act
  x = layers.MaxPooling2D(pool_size=(2,2), name='pool1_pool')(x)
  x = layers.Dropout(config["dropout"])(x)
  x = stack_fn(x)
  
  x = layers.BatchNormalization(epsilon=1.001e-5, name='post_bn')(x)
  x = layers.Activation('relu', name='post_relu')(x)
  x = layers.GlobalAveragePooling2D()(x)
  #x = layers.MaxPooling2D(pool_size=(2,2))(x)
  #x = layers.Flatten()(x)

  #classified
  digit1 = layers.Dense(36, name='digit1', activation='softmax', kernel_initializer = initializer,kernel_regularizer = regularizer)(x)
  digit2 = layers.Dense(36, name='digit2', activation='softmax', kernel_initializer = initializer,kernel_regularizer = regularizer)(x)
  digit3 = layers.Dense(36, name='digit3', activation='softmax', kernel_initializer = initializer,kernel_regularizer = regularizer)(x)
  digit4 = layers.Dense(36, name='digit4', activation='softmax', kernel_initializer = initializer,kernel_regularizer = regularizer)(x)
  digit5 = layers.Dense(36, name='digit5', activation='softmax', kernel_initializer = initializer,kernel_regularizer = regularizer)(x)
  digit6 = layers.Dense(36, name='digit6', activation='softmax', kernel_initializer = initializer,kernel_regularizer = regularizer)(x)
  modelt = tf.keras.models.Model(inputs=img_input,outputs = [digit1 , digit2, digit3, digit4, digit5, digit6], name = config["name"])
  

  

  modelt.compile(
    loss=[
        tf.keras.losses.CategoricalCrossentropy(),
        tf.keras.losses.CategoricalCrossentropy(),
        tf.keras.losses.CategoricalCrossentropy(),
        tf.keras.losses.CategoricalCrossentropy(),
        tf.keras.losses.CategoricalCrossentropy(),
        tf.keras.losses.CategoricalCrossentropy(),
    ],
    optimizer=tf.keras.optimizers.Adam(learning_rate=config["LR"], beta_1=0.9), 
    metrics=['accuracy'])
  return modelt

In [0]:
  #railway model add 1 256 layer
def railway_model(config=config_default):
    initializer = tf.keras.initializers.he_normal( seed = 3)
    alpha = config["weight_regu"]  # weight decay coefficient
    regularizer = tf.keras.regularizers.l2(alpha)
    dropout_rate =config["dropout"]
    inn = Input((60, 200, 3))
    out = inn
    out = Conv2D(filters=36, kernel_size=(3, 3), padding='same', activation='relu',kernel_initializer = initializer,kernel_regularizer = regularizer)(out)
    out = Conv2D(filters=36, kernel_size=(3, 3), activation='relu',kernel_initializer = initializer,kernel_regularizer = regularizer)(out)
    out = BatchNormalization()(out)
    out = MaxPooling2D(pool_size=(2, 2))(out)
    out = Dropout(dropout_rate)(out)
    out = Conv2D(filters=72, kernel_size=(3, 3), padding='same', activation='relu',kernel_initializer = initializer,kernel_regularizer = regularizer)(out)
    out = Conv2D(filters=72, kernel_size=(3, 3), activation='relu',kernel_initializer = initializer,kernel_regularizer = regularizer)(out)
    out = BatchNormalization()(out)
    out = MaxPooling2D(pool_size=(2, 2))(out)
    out = Dropout(dropout_rate)(out)
    out = Conv2D(filters=144, kernel_size=(3, 3), padding='same', activation='relu',kernel_initializer = initializer,kernel_regularizer = regularizer)(out)
    out = Conv2D(filters=144, kernel_size=(3, 3), activation='relu',kernel_initializer = initializer,kernel_regularizer = regularizer)(out)
    out = BatchNormalization()(out)
    out = MaxPooling2D(pool_size=(2, 2))(out)
    out = Dropout(dropout_rate)(out)
    out = Conv2D(filters=288, kernel_size=(3, 3), padding='same',activation='relu',kernel_initializer = initializer,kernel_regularizer = regularizer)(out)
    out = Conv2D(filters=288, kernel_size=(3, 3), activation='relu',kernel_initializer = initializer,kernel_regularizer = regularizer)(out)
    out = BatchNormalization()(out)
    out = MaxPooling2D(pool_size=(2, 2))(out)
    out = Flatten()(out)
    sep = Dropout(dropout_rate)(out)
    dig1 = Dense(36, name='digit1', activation='softmax',kernel_initializer = initializer,kernel_regularizer = regularizer)(sep)
    dig2 = Dense(36, name='digit2', activation='softmax',kernel_initializer = initializer,kernel_regularizer = regularizer)(sep)
    dig3 = Dense(36, name='digit3', activation='softmax',kernel_initializer = initializer,kernel_regularizer = regularizer)(sep)
    dig4 = Dense(36, name='digit4', activation='softmax',kernel_initializer = initializer,kernel_regularizer = regularizer)(sep)
    dig5 = Dense(36, name='digit5', activation='softmax',kernel_initializer = initializer,kernel_regularizer = regularizer)(sep)
    dig6 = Dense(36, name='digit6', activation='softmax',kernel_initializer = initializer,kernel_regularizer = regularizer)(sep)
    model = tf.keras.models.Model(inputs=inn, outputs=[dig1, dig2,dig3,dig4,dig5,dig6],name=config["name"])
    model.compile(
        loss=[
            tf.keras.losses.CategoricalCrossentropy(),
            tf.keras.losses.CategoricalCrossentropy(),
            tf.keras.losses.CategoricalCrossentropy(),
            tf.keras.losses.CategoricalCrossentropy(),
            tf.keras.losses.CategoricalCrossentropy(),
            tf.keras.losses.CategoricalCrossentropy(),
        ],
        optimizer=tf.keras.optimizers.Adam(learning_rate=config["LR"], beta_1=0.9),
        metrics=['accuracy'])
    model.summary()
    return model

# Fit model

In [0]:
def trainModel(img_train, lb_train, img_test, lb_test,callB,epochs=100,config = config_default):
  history = model.fit(
        img_train,
        lb_train,
        batch_size = config["batch"],
        shuffle = True,
        validation_data=(img_test, lb_test),
        epochs=epochs,
        verbose=1,
        callbacks = callB
        )

In [0]:
def switch_to_SGD():
  model.compile(
    loss=[
        tf.keras.losses.CategoricalCrossentropy(),
        tf.keras.losses.CategoricalCrossentropy(),
        tf.keras.losses.CategoricalCrossentropy(),
        tf.keras.losses.CategoricalCrossentropy(),
        tf.keras.losses.CategoricalCrossentropy(),
        tf.keras.losses.CategoricalCrossentropy()
    ],
    optimizer=tf.keras.optimizers.SGD(learning_rate=0.0001, momentum=0.9), 
    metrics=['accuracy'])

In [0]:
class myCallback(tf.keras.callbacks.Callback):
  def __init__(self, config=config_default,**kwargs):
    self.config = config
    self.best = 0
    self.epochs = 0
    self.wait = 0
    self.reduce_once = False
    self.patient = 8  #wait 8 epochs for early stopping
  def on_epoch_end(self, epoch, logs=None):
    self.epochs+=1
    b= np.prod([logs['digit1_accuracy'],
              logs['digit2_accuracy'],
              logs['digit3_accuracy'],
              logs['digit4_accuracy'],
              logs['digit5_accuracy'],
              logs['digit6_accuracy'],
              ])
    a= np.prod([logs['val_digit1_accuracy'],
              logs['val_digit2_accuracy'],
              logs['val_digit3_accuracy'],
              logs['val_digit4_accuracy'],
              logs['val_digit5_accuracy'],
              logs['val_digit6_accuracy'],
              ])
    wandb.log({"loss":logs["loss"],"val_loss" : logs["val_loss"],"accuracy": b, "val_accuracy":a, "epoch": self.epochs})
    print("\nepoch: {}, loss: {:.4f}, val_loss: {:.4f}, accuracy: {:.5f}, val_accuracy: {:.5f}".format(self.epochs,logs["loss"],logs["val_loss"],b,a))
    if a > self.best:
      print("                            ***** accuracy improved from {:.5f} to {:.5f}!! *****".format(self.best,a))
      print("Model saved to "+filepath+self.config["name"]+".hdf5\n" )
      self.best = a
      self.model.save(filepath+self.config["name"]+".hdf5")
      self.wait = 0
      wandb.run.summary["best_val_accuracy"] = a
      wandb.run.summary["best_epoch"] = self.epochs
    else:
      if self.wait>=self.patient:   #switch to SGD or reduce lr if already on SGD 
        if hasattr(self.model.optimizer,'momentum') and not self.reduce_once:     #reduce lr on SGD if not done it once yet
          self.model.optimizer.lr=self.model.optimizer.lr/2
          print("\n\n Change leraning rate to {}, continued ... \n".format(self.model.optimizer.lr))
          self.wait=0
          self.reduce_once=True
        else:
          self.model.stop_training = True
          self.wait=0
        print("Model is not learning, training stop at {}".format(epoch))
      print("Model accuracy did not improve from {:.4f}\n".format(self.best))
      self.wait+=1
      
class Switch_SGD_callback(tf.keras.callbacks.Callback):
  def __init__(self,on_train_end, **kwargs):
    self.do_on_train_end=on_train_end
  def on_train_end(self, logs=None):
      self.do_on_train_end()

def do_after_train():
    print("\n ******** Switching to SGD!! **********")
    switch_to_SGD()
    trainModel(img_train, lb_train, img_test, lb_test,[myCB])

In [0]:
model = build_ResNet_Model(config=myconfig)
model.summary()

In [0]:
#initialize callback object
myCB = myCallback(config=config)

data 01目前最好的:

val_accuracy: 98.11%

dev01: 97.72%

conv 3x3: 32,32,64,64,128,128,256,256

weight_regu     0.005

dropout         0.3

batch           256

備註: batch沒試過其他的，val和test有差0.4%可能是val不夠具有代表性，實際上存的不是最好的


In [0]:
#train for 100 epochs, switch to SGD if accuracy does not increase for 10 epochs, abort training it happens again

myconfig = {
    "stack" : "conv 3x3: 32   ,block2 : 32,64,128,256",
    "weight_regu" : 1e-4,
    "dropout" : 0.3,
    "preact" : True,
    "name" : "railway_ResNet_4",
    "output_WR" : True,
    "total_params" : 3727704,
    "opt_schedule" : "1e-3 adam + 1e-4 SGD",
    "batch" : 64,
    "GlobalAvg":True,
    "preact":"include shortcut"
}
wandb.init(project="my-project-railway",reinit=True,name=config["name"],config=myconfig) 

#choose model
model = build_ResNet_Model(config=myconfig)
#ini call back
myCB = myCallback(config = myconfig)
trainModel(img_train, lb_train, img_test, lb_test,[myCB,Switch_SGD_callback(do_after_train)],config = myconfig)

In [0]:
#manual switch to SGD
switch_to_SGD()
trainModel(img_train, lb_train, img_test, lb_test,[myCB])

In [0]:
#Manual save
filepath=""
model.save(filepath)